In [16]:
import boto3
import pandas as pd
client = boto3.client('rekognition')
s3_resource = boto3.resource('s3')
my_bucket_origs = s3_resource.Bucket('orig-imgs-parmigiana')
my_bucket_transformed = s3_resource.Bucket('transformed-imgs-parmigiana')
summaries_origs = my_bucket_origs.objects.all() ## contains all original images

In [17]:
def create_dict(category, image_type, target): 
    summaries_transformed = my_bucket_transformed.objects.filter(Prefix=image_type) #contains images under certain type
    confid_dict = {"Image" : [], category :[], "Label" : [], "Confidence" :[]}
    for image in summaries_transformed:
        name = image.key.split('-')[1].split('.j')[0]
        if name.split(':')[0] == category.lower():
            ind = -1
            response = client.detect_labels(Image={'S3Object':{'Bucket':"transformed-imgs-parmigiana" ,'Name':image.key}}, MaxLabels=10)
            confidence = response['Labels'][0]['Confidence']
            for obj in response['Labels']: 
                if(obj['Name'] == target): 
                    ind = response['Labels'].index(obj)
                    break
            if ind != -1:
                confid_dict["Image"].append(image.key.split('-')[0])
                confid_dict[category].append(name.split(':')[1])
                confid_dict["Label"].append(response['Labels'][ind]['Name'])
                confid_dict["Confidence"].append(round(confidence,3))
    return confid_dict

In [ ]:
ind_remove = confid_dict['Image'].index('crosswalk8') #value to be removed if any inconsistencies

In [18]:
def create_df(confid_dict):
    df = pd.DataFrame(confid_dict)
    return df

In [ ]:
df = df.drop(ind_remove) #remove from df

In [34]:
#iterate through all categories and variable changes (brightness/sharpness/contrast) and known targets
categories = {'crosswalk': 'Road', 'food' : 'Food'} #categories and targets
variables = {'Bright', 'Sharp', 'Contrast'}
for i in categories.keys(): 
    for j in variables: 
        new_dict = create_dict(j, i, categories.get(i))
        new_df = create_df(new_dict)
        #remove imgs 1, 2, 5, 6, 8 from crosswalk
        if i == "crosswalk" and j != 'Sharp': 
            #values to be removed due to inconsistencies 
            #unequal distribution for data analysis in lower and upper levels
            new_df = new_df.drop(new_dict['Image'].index('crosswalk8')) #removed
            if j == 'Contrast': 
                new_df = new_df.drop(new_dict['Image'].index('crosswalk1')) #removed
        #remove imgs 1, 6 from food
        if i == "food": 
            if j == 'Bright': 
                new_df = new_df.drop([0,1,52,53])
            if j == 'Contrast':
                new_df = new_df.drop([0,1,2])
#             remove from df pictures 1 and 6 due to inconsistencies
#             unequal distribution for data analysis in lower and upper levels
        new_df = new_df.set_index("Image").sort_values(by=['Image', j])
        new_df.to_csv(i + '-' + j.lower() + '.csv')